In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


import matplotlib.pyplot as plt

from src.shared.sides import sphered_vertices
# for i in range(10000):
#     #compute the predictions    
#     preds = model(inputs) 
#     #compute the loss   
#     loss = loss_fn(preds, targets) 
#     # calculate the gradients   
#     loss.backward()    
#     with torch.no_grad():        
#         weight -= weight.grad * 1e-7        
#         bias -= bias.grad * 1e-7        
#         weight.grad.zero_()        
#         bias.grad.zero_()

In [9]:
# Need to fix diagonals
# Probably multiply  with conv
# 0 1 0
# 1 0 1
# 0 1 0


mxd, mnd, hd = max_dist(torch.randn(1, 3, 2, 2))
mxd, mnd

(tensor([[[3.6261, 3.6261],
          [3.6261, 3.6261]]]), tensor([[[2.2465, 2.2465],
          [2.2465, 2.2465]]]))

In [10]:
hd.shape

torch.Size([1, 2, 2])

In [6]:
# hd
# 0 1 2 3
# 1
# 2
# 3
v = sphered_vertices(2, 0.5)
v.shape

torch.Size([6, 3, 2, 2])

In [8]:
mxd, mnd = max_dist(v)
print(mxd)
print(mnd)

tensor([[[0.4082, 0.4082],
         [0.4082, 0.4082]],

        [[0.4082, 0.4082],
         [0.4082, 0.4082]],

        [[0.4082, 0.4082],
         [0.4082, 0.4082]],

        [[0.4082, 0.4082],
         [0.4082, 0.4082]],

        [[0.4082, 0.4082],
         [0.4082, 0.4082]],

        [[0.4082, 0.4082],
         [0.4082, 0.4082]]])
tensor([[[0.4082, 0.4082],
         [0.4082, 0.4082]],

        [[0.4082, 0.4082],
         [0.4082, 0.4082]],

        [[0.4082, 0.4082],
         [0.4082, 0.4082]],

        [[0.4082, 0.4082],
         [0.4082, 0.4082]],

        [[0.4082, 0.4082],
         [0.4082, 0.4082]],

        [[0.4082, 0.4082],
         [0.4082, 0.4082]]])


In [13]:
conv = nn.Conv2d(3, 3, 3, 
            stride=1, padding=0, bias=False, groups=3)

w = torch.tensor([[ 0., 1., 0.],
                  [ 1., 1., 1.],
                  [ 0., 1., 0.],])
for _, f in conv.named_parameters():           
    f.data.copy_(w)
conv.weight

In [14]:
class Distances(nn.Module):
    def __init__(self):
        super(Distances, self).__init__()
        self.conv = nn.Conv2d(3, 3, 3, stride=1,
            padding=1, bias=False, groups=3)
        self.weights_init()

    
    
    def forward(self, x):
        return self.seq(x).abs().mean()

    def weights_init(self):
        w = torch.tensor([[ 0., 1., 0.],
                          [ 1., 1., 1.],
                          [ 0., 1., 0.],])
        for _, f in self.named_parameters():           
            f.data.copy_(w)


Parameter containing:
tensor([[[[0., 1., 0.],
          [1., 1., 1.],
          [0., 1., 0.]]],


        [[[0., 1., 0.],
          [1., 1., 1.],
          [0., 1., 0.]]],


        [[[0., 1., 0.],
          [1., 1., 1.],
          [0., 1., 0.]]]], requires_grad=True)

In [17]:
conv = nn.Conv2d(3, 3, kernel_size=(3, 1), 
            stride=1, padding=0, bias=False, groups=3)
conv.weight.shape

torch.Size([3, 1, 3, 1])

In [31]:
def distances(x):
    with torch.no_grad():
        n = x.size(-1)
        hd = x[:, :, 1:, :] - x[:, :, :-1, :]
        hd = hd.pow(2).sum(dim=1).sqrt()
        hd = torch.cat((hd[:, 0:1, :], hd), dim=1)        
        hdmx = F.max_pool2d(hd, kernel_size=(3, 1), stride=1, padding=(1, 0))
        hdmn = -F.max_pool2d(-hd, kernel_size=(3, 1), stride=1, padding=(1, 0))

        vd = x[:, :, :, 1:] - x[:, :, :, :-1]
        vd = vd.pow(2).sum(dim=1).sqrt()
        vd = torch.cat((vd[:, :, 0:1], vd), dim=2)        
        vdmx = F.max_pool2d(vd, kernel_size=(1, 3), stride=1, padding=(0, 1))
        vdmn = -F.max_pool2d(-vd, kernel_size=(1, 3), stride=1, padding=(0, 1))

        mxd = torch.where(hdmx > vdmx, hdmx, vdmx)
        mnd = torch.where(hdmn < vdmn, hdmn, vdmn)
        return mxd, mnd
    
mxd, mnd = distances(torch.randn(1, 3, 4, 4))
#mxd, mnd, hd = max_dist(torch.randn(1, 3, 2, 2))
#mxd, mnd    
print(mxd)
print(mnd)

tensor([[[3.6284, 3.6284, 3.6284, 3.4122],
         [3.4133, 3.0668, 3.7784, 3.7784],
         [3.4133, 3.0668, 3.2177, 2.7116],
         [3.4133, 3.0668, 3.2177, 2.7116]]])
tensor([[[1.8303, 2.4891, 1.1815, 1.1815],
         [1.8303, 2.4609, 2.0027, 2.0350],
         [1.8303, 1.4759, 1.5877, 1.5794],
         [1.8711, 1.4759, 1.8711, 1.5794]]])


In [24]:
F.max_pool2d(hd, kernel_size=(3, 1), stride=1, padding=(1, 0)).shape

torch.Size([1, 2, 2])

In [32]:
import numpy as np

a = np.array([32.49, -39.96,-3.86])
b = np.array([31.39, -39.28, -4.66])
c = np.array([31.14, -38.09,-4.49])

ba = a - b
bc = c - b

cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
angle = np.arccos(cosine_angle)

np.degrees(angle)

120.52106374124477

In [35]:

a = torch.tensor([32.49, -39.96,-3.86])
b = torch.tensor([31.39, -39.28, -4.66])
c = torch.tensor([31.14, -38.09,-4.49])

ba = a - b
bc = c - b

cosine_angle = torch.dot(ba, bc) / (torch.linalg.norm(ba) * torch.linalg.norm(bc))
angle = torch.arccos(cosine_angle)

torch.rad2deg(angle)

tensor(120.5211)

In [36]:
a, b, c = torch.randn(6, 3, 4, 4), torch.randn(6, 3, 4, 4), torch.randn(6, 3, 4, 4)
ba = a - b
bc = c - b
ba.shape,  bc.shape

(torch.Size([6, 3, 4, 4]), torch.Size([6, 3, 4, 4]))

In [38]:
dot = ba * bc
dot.shape

torch.Size([6, 3, 4, 4])

In [44]:

cos_angle.shape

torch.Size([6, 3, 4, 4])

In [46]:
angle = torch.arccos(cos_angle)
torch.rad2deg(angle)

tensor([[[[ 54.5833,  43.7095,  22.2166, 107.4582],
          [ 70.2230,  92.3000, 115.3340,  85.8464],
          [110.8317,  94.3133,  90.8866,  76.6120],
          [ 81.1190,  52.5785,  81.2940,  82.9476]],

         [[107.9911,  78.7944,  86.5459,  84.1042],
          [ 62.4340,  52.7505,  90.1965,  41.7930],
          [ 68.5803, 126.5247, 113.9567,  90.0513],
          [ 82.5215,  82.4468,  49.2231,  74.6128]],

         [[ 91.0299,  90.3734,  89.5175,  69.1811],
          [ 91.8565,  77.1799,  81.8335,  90.1359],
          [ 98.2441, 100.1359,  58.1367, 118.7322],
          [ 62.6941,  80.1737,  96.6341,  58.6585]]],


        [[[ 82.1992,  86.1070, 108.4879,  85.3625],
          [ 91.5687, 116.9693,  91.3344,  35.5550],
          [ 97.6212,  67.7274,  98.2771, 118.1585],
          [124.0369,  92.6306,  91.1320, 117.8011]],

         [[ 82.8775,  84.0063,  99.0044,  35.1682],
          [103.5373,  94.6409,  26.6277,  88.6885],
          [127.0545,  53.1233,  78.5858,  79.6933],
  

In [56]:

def get_angles(x):
    def directional_angles(a, b, c):
        ba = a - b
        bc = c - b
        dot = (ba * bc)
        norm = torch.linalg.norm(ba, dim=1, keepdim=True) * torch.linalg.norm(bc, dim=1, keepdim=True)
        cos_angle = dot / norm
        angle = torch.arccos(cos_angle)
        return torch.rad2deg(angle)
    
    a = x[:, :, 0:-2, :]
    b = x[:, :, 1:-1, :]
    c = x[:, :, 2:, :]
    ha = directional_angles(a, b, c)
    a = x[:, :, :, 0:-2]
    b = x[:, :, :, 1:-1]
    c = x[:, :, :, 2:]
    va = directional_angles(a, b, c)
    
    return ha, va

ha, va = get_angles(x)#.shape
ha.shape, va.shape

(torch.Size([6, 3, 2, 4]), torch.Size([6, 3, 4, 2]))